In [1]:
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Path to your dataset
dataset_path = '/data'
class_names = ['happy', 'sad']  # replace with your class names

# Initialize data and labels lists
data = []
labels = []

# Load images
for class_index, class_name in enumerate(class_names):
    folder_path = os.path.join('data', class_name)
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):  # Check for valid image formats
            print(f"Skipping non-image file: {image_name}")
            continue
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")  # Print an error message
            continue
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        image = cv2.resize(image, (128, 128))  # Resize images to 128x128 pixels
        data.append(image)
        labels.append(class_index)

# Convert lists to numpy arrays
data = np.array(data, dtype="float") / 255.0  # Normalize pixel values
data = np.expand_dims(data, axis=-1)  # Add a channel dimension
labels = np.array(labels)

# One-hot encode the labels
labels = to_categorical(labels, num_classes=2)

# Split the dataset into training and testing sets
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data\\happy'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", input_shape=(128, 128, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="softmax"))  # For binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=15, batch_size=32)

# Save the trained model
model.save('face_recognition_model.h5')

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(testX, testY)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

# Load a single image and preprocess it
image_path = '/content/Sad_Person.jpeg'
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (128, 128))
image = np.expand_dims(image, axis=-1)  # Add channel dimension
image = np.expand_dims(image, axis=0)   # Add batch dimension
image = image.astype("float") / 255.0   # Normalize pixel values

# Predict the class
prediction = model.predict(image)
predicted_class = class_names[np.argmax(prediction)]
print(f"Predicted Class: {predicted_class}")

img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
cv2_imshow(img)